### Q1. Install MLflow
To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 1.26.1


### Q2. Download and preprocess the data
We'll use the Green Taxi Trip Records dataset.

Download the data for January, February and March 2021 in parquet format from here.

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will

load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
fit a DictVectorizer on the training set (January 2021 data),
save the preprocessed datasets and the DictVectorizer to disk
Your task is to download the datasets and then execute this command:

python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
Tip: go to 02-experiment-tracking/homework/ folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.

How many files were saved to OUTPUT_FOLDER?

1 <br>
3 <br>
4 <br>
7 <br>

In [8]:
pwd()


'/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/homework'

In [9]:
!mkdir data
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/green_tripdata_2021-01.parquet -P data/
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/green_tripdata_2021-02.parquet -P data/
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/green_tripdata_2021-03.parquet -P data/

--2022-06-02 14:10:14--  https://nyc-tlc.s3.amazonaws.com/trip+data/green_tripdata_2021-01.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.226.193
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.226.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2021-01.parquet’

green_tripdata_2021 100%[===================>]   1.27M  --.-KB/s    in 0.1s    

2022-06-02 14:10:14 (10.9 MB/s) - ‘data/green_tripdata_2021-01.parquet’ saved [1333519/1333519]

--2022-06-02 14:10:16--  https://nyc-tlc.s3.amazonaws.com/trip+data/green_tripdata_2021-02.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.128.1
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.128.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145679 (1.1M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_20

In [10]:
!mkdir processed_data


In [11]:
!ls data


green_tripdata_2021-01.parquet	green_tripdata_2021-03.parquet
green_tripdata_2021-02.parquet


In [12]:
!ls

data	preprocess_data.py  register_model.py  week2_hw.ipynb
hpo.py	processed_data	    train.py


In [14]:
!pwd

/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/homework


In [16]:
!python preprocess_data.py --raw_data_path data/ --dest_path processed_data/


In [17]:
!ls processed_data/

dv.pkl	test.pkl  train.pkl  valid.pkl


### Q3. Train a model with autolog
We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

19 <br>
17 <br>
10 <br>
20 <br>

In [18]:
!python train.py --data_path processed_data/


2022/06/02 14:16:46 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/06/02 14:16:46 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

#### output
Parameters (17) <br>
Name	Value <br>
bootstrap	True <br>
ccp_alpha	0.0 <br>
criterion	squared_error <br>
max_depth	10 <br>
max_features	1.0 <br>
max_leaf_nodes	None <br>
max_samples	None <br>
min_impurity_decrease	0.0 <br>
min_samples_leaf	1 <br>
min_samples_split	2 <br>
min_weight_fraction_leaf	0.0 <br>
n_estimators	100 <br>
n_jobs	None <br>
oob_score	False <br>
random_state	0 <br>
verbose	0 <br>
warm_start	False <br>

#### Q4. Launch the tracking server locally
We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to

launch the tracking server on your local machine
select a SQLite db for the backend store and a folder called artifacts for the artifacts store
You should keep the tracking server running to work on the next two exercises that use the server.

In addition to backend-store-uri, what else do you need to pass to properly configure the server?

default-artifact-root <br>
serve-artifacts <br>
artifacts-only <br>
artifacts-destination <br>

In [20]:
import mlflow
mlflow.end_run()


In [23]:
!ps -ef| grep mlfow

ubuntu     23651   23463  0 14:25 pts/3    00:00:00 /bin/bash -c ps -ef| grep mlfow
ubuntu     23653   23651  0 14:25 pts/3    00:00:00 grep mlfow


In [25]:
!mlflow server --backend-store-uri sqlite:///mlflow_week2.db --default-artifact-root ./artifacts_local

2022/06/02 14:25:33 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/06/02 14:25:33 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [26]:
!ps -A | grep gunicorn

  11811 ?        00:00:25 gunicorn
  15527 ?        00:00:03 gunicorn
  15528 ?        00:00:03 gunicorn
  15529 ?        00:00:03 gunicorn
  15530 ?        00:00:04 gunicorn


In [28]:
!kill -9 11811 15527 15528 15529 15530

In [32]:
!mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts_local

[2022-06-02 15:08:08 +0000] [24508] [INFO] Starting gunicorn 20.1.0
[2022-06-02 15:08:08 +0000] [24508] [INFO] Listening at: http://127.0.0.1:5000 (24508)
[2022-06-02 15:08:08 +0000] [24508] [INFO] Using worker: sync
[2022-06-02 15:08:08 +0000] [24509] [INFO] Booting worker with pid: 24509
[2022-06-02 15:08:08 +0000] [24510] [INFO] Booting worker with pid: 24510
[2022-06-02 15:08:08 +0000] [24511] [INFO] Booting worker with pid: 24511
[2022-06-02 15:08:08 +0000] [24512] [INFO] Booting worker with pid: 24512


#### default-artifact-root is the optional parameter.

### Q5. Tune the hyperparameters of the model
Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using hyperopt.

We have prepared the script hpo.py for this exercise.

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

Important: don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

the list of hyperparameters that are passed to the objective function during the optimization.
the RMSE obtained on the validation set (February 2021 data).
What's the best validation RMSE that you got?

6.128 <br>
6.628 <br>
7.128 <br>
7.628 <br>


(exp-tracking-env) ubuntu@ip-172-31-4-164:~/mlops-zoomcamp/02-experiment-tracking/homework$ python hpo.py --data_path processed_data/
100%|█████████████████████████████████████████████████████████████████████████████████████████| 50/50 [08:44<00:00, 10.48s/trial, best loss: 6.6284257482044735]
(exp-tracking-env) ubuntu@ip-172-31-4-164:~/mlops-zoomcamp/02-experiment-tracking/homework$ 

### Q6. Promote the best model to the model registry
The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE. <br>
Tip 2: to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!). <br>

What is the test RMSE of the best model? <br>

6.1 <br>
6.55 <br>
7.93 <br>
15.1 <br>

In [ ]:
!python register_model.py --data_path processed_data/


In [ ]:
(exp-tracking-env) ubuntu@ip-172-31-4-164:~/mlops-zoomcamp/02-experiment-tracking/homework$ python register_model.py --data_path processed_data/
2022/06/02 17:22:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
valid_rmse:6.6284257482044735, test_rmse:6.547886185595423
valid_rmse:6.629728007710133, test_rmse:6.549816636724069
valid_rmse:6.629851022038263, test_rmse:6.549081827157288
valid_rmse:6.629851022038263, test_rmse:6.549081827157288
valid_rmse:6.629893219637507, test_rmse:6.548852722401935
Best Model URI:runs/24735831704841bcbbdc1ded69fe1387/models
Successfully registered model 'RandomForestRegressionModel'.
2022/06/02 17:24:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestRegressionModel, version 1
Created version '1' of model 'RandomForestRegressionModel'.
(exp-tracking-env) ubuntu@ip-172-31-4-164:~/mlops-zoomcamp/02-experiment-tracking/homework$ 